list google contact?



In [ ]:
var google = require('googleapis');
var people = google.people('v1');
var importer = require('../Core');
var getOauthClient = importer.import('import google calendar api');

var options = {
    scopes: [
        'https://www.googleapis.com/auth/contacts'
    ]
};

function findMatch(arr, contact) {
    return arr.filter(c => (
        typeof c.names !== 'undefined'
        && typeof contact.displayName !== 'undefined'
        && c.names.filter(n => n.displayName.match(new RegExp(contact.displayName, 'ig'))).length > 0)
        || (
        typeof c.emailAddresses !== 'undefined'
        && typeof contact.emailAddress !== 'undefined'
        && c.emailAddresses.filter(e => e.value.match(new RegExp(contact.emailAddress, 'ig'))).length > 0)
                      );
}

function googlePromise(func) {
    return new Promise((resolve, reject) => func((err, response) => {
        if (err) reject(err);
        try {
        } catch (e) {
            reject(e);
        }
        setTimeout(() => resolve(response), 100);
    })).catch(e => console.log(e));
};

var contactCache = [];
function listContacts(contact, done = false) {
    // return matching contacts or empty if there are no more pages
    return googlePromise(
        people.people.connections.list.bind(people.people.connections, {
            resourceName: 'people/me',
            personFields: 'emailAddresses,names,birthdays,phoneNumbers,memberships',
            auth: options.auth,
            pageToken: options.pageToken
        }))
        .then(r => {
            options.pageToken = r.nextPageToken;
            done = !options.pageToken;
            return r.connections;
        })
        .then(r => {
            for (var c of r) {
                contactCache.push(c);
            }
            if (done) {
                return findMatch(contactCache, contact);
            }
            return listContacts(contact, done);
        })
        .catch(e => console.log(e))
};

function getContacts(contact) {
    return getOauthClient(options)
        .then(() => {
            if (contactCache.length > 0) {
                return findMatch(contactCache, contact);
            }
            options.nextPageToken = null;
            contactCache = [];
            return listContacts(contact);
        })
        .catch(e => console.log(e))
};
module.exports = getContacts;


google contact settings?



In [2]:
var google = require('googleapis');
var people = google.people('v1');
var importer = require('../Core');
var getContacts = importer.import('list google contacts');
var getOauthClient = importer.import('import google calendar api');

var options = {
    scopes: [
        'https://www.googleapis.com/auth/contacts'
    ]
};

function googlePromise(func) {
    return new Promise((resolve, reject) => func((err, response) => {
        if (err) reject(err);
        try {
        } catch (e) {
            reject(e);
        }
        setTimeout(() => resolve(response), 100);
    })).catch(e => console.log(e));
};

function mapMembership(memberships) {
    return {
        'frequency': ((memberships.filter(m => m.contactGroupMembership.contactGroupLabel === 'Weekly'
                                        ||  m.contactGroupMembership.contactGroupLabel === 'Yearly'
                                        ||  m.contactGroupMembership.contactGroupLabel === 'Monthly')[0] || {})
            .contactGroupMembership || {}).contactGroupLabel,
        'circles': memberships.filter(m => m.contactGroupMembership.contactGroupLabel === 'YouTube'
                                        || m.contactGroupMembership.contactGroupLabel === 'Coworkers'
                                        || m.contactGroupMembership.contactGroupLabel === 'Family'
                                        || m.contactGroupMembership.contactGroupLabel === 'friends'
                                        || m.contactGroupMembership.contactGroupLabel === 'Recruiters'
                                        || m.contactGroupMembership.contactGroupLabel === 'E-Level'
                                        || m.contactGroupMembership.contactGroupLabel === 'A-Level'
                                        || m.contactGroupMembership.contactGroupLabel === 'C-Level'
                                        || m.contactGroupMembership.contactGroupLabel === 'Inner Circle'
                                        || m.contactGroupMembership.contactGroupLabel === 'Outer Circle')
            .map(c => c.contactGroupMembership.contactGroupLabel),
        'preferred': memberships.filter(m => m.contactGroupMembership.contactGroupLabel === 'LinkedIn'
                                        || m.contactGroupMembership.contactGroupLabel === 'Facebook'
                                        || m.contactGroupMembership.contactGroupLabel === 'Email'
                                        || m.contactGroupMembership.contactGroupLabel === 'Mobile'
                                        || m.contactGroupMembership.contactGroupLabel === 'Skype'
                                        || m.contactGroupMembership.contactGroupLabel === 'Slack')
            .map(c => c.contactGroupMembership.contactGroupLabel)
    }
}

function getSettings(name) {
    var groups;
    return getOauthClient(options)
        .then(() => googlePromise(
            people.contactGroups.list.bind(people.contactGroups.list, {
                auth: options.auth,
                pageToken: options.pageToken
            })).catch(e => console.log(e)))
        .then(g => groups = g.contactGroups)
        .then(() => {
            const options = {};
            if(name.indexOf('@') > -1) {
                options.emailAddress = name.split(/\s/).join('.*');
            } else {
                options.displayName = name.split(/\s/).join('.*');
            }
            return getContacts(options);
        })
        .then(contacts => contacts.map(c => {
            return Object.assign(c, {
                memberships: c.memberships.map(m => Object.assign(m, {
                    contactGroupMembership: Object.assign(m.contactGroupMembership, {
                        contactGroupLabel: (groups.filter(g => (g.resourceName || '').indexOf(m.contactGroupMembership.contactGroupId) > -1)[0] || {}).name
                    })
                }))
            });
        }))
        .then(contacts => mapMembership(contacts[0].memberships))
        .catch(e => console.log(e))
}
module.exports = getSettings;


[Function: getSettings]

In [8]:
var importer = require('../Core');
var getSettings = importer.import('google contact settings');

$$.async();
getSettings('megamindbrian@gmail.com')
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e))


{ resourceName: 'people/c294269373163842527',
  etag: '%EgcBAgcJCzcuGg0BAgMEBQYHCAkKCwwNIgw4N2lOU1RMcncwST0=',
  names: 
   [ { metadata: [Object],
       displayName: 'Brian Cullinan',
       familyName: 'Cullinan',
       givenName: 'Brian',
       displayNameLastFirst: 'Cullinan, Brian' },
     { metadata: [Object],
       displayName: 'Brian Cullinan',
       familyName: 'Cullinan',
       givenName: 'Brian',
       displayNameLastFirst: 'Cullinan, Brian' } ],
  emailAddresses: 
   [ { metadata: [Object],
       value: 'bjcullinan@gmail.com',
       type: 'other',
       formattedType: 'Other' },
     { metadata: [Object],
       value: 'megamindbrian@gmail.com',
       type: 'other',
       formattedType: 'Other' } ],
  phoneNumbers: 
   [ { metadata: [Object],
       value: '+1 480-466-0856',
       canonicalForm: '+14804660856' } ],
  memberships: 
   [ { metadata: [Object], contactGroupMembership: [Object] },
     { metadata: [Object], contactGroupMembership: [Object] },
     {

{ frequency: undefined, circles: [ 'YouTube' ] }